### Imports

In [2]:
%pylab inline

import requests
import csv
import time
import hashlib
import random
import cookielib
import sys
import re
import urllib
from  urllib2 import *
import getpass
import pandas as pd
import numpy as np

#from ghost import Ghost 
from lxml import html
from unidecode import unidecode

Populating the interactive namespace from numpy and matplotlib


### Auxiliar Methods

In [22]:
### Not used
def authenticate():
    email = raw_input("Google username: ")
    password = getpass.getpass("Password: ")

    LOGIN_URL = 'https://accounts.google.com/ServiceLogin?service=grandcentral'
    AUTH_URL = 'https://accounts.google.com/ServiceLoginAuth?service=grandcentral'
    opener = build_opener(HTTPCookieProcessor()) 

    # Find GALX value
    login_page_contents = opener.open(LOGIN_URL).read()
    galx = re.search(r"name=\"GALX\"\s+type=\"hidden\"\s+value=\"(.+)\"", login_page_contents, re.IGNORECASE).group(1)
    # print galx 

    login_data = urllib.urlencode({
        'Email': email,
        'Passwd': password,
        'GALX': galx,
    })

    opener.open(AUTH_URL, login_data)

# Sleep time to avoid hard hitting Google Scholar
def sleep_and_build_opener():
    time.sleep(MIN_SLEEP + (random.random() * MAX_RANDOM_MINUTES))  
    global opener # TODO maybe is not necessary build every time (must test against GS blocks)
    opener = build_opener(HTTPCookieProcessor()) 
    
def open_allowed_page():
    time.sleep(MIN_SLEEP)
    opener.open('http://scholar.google.com.br/citations?user=JMkfK0sAAAAJ&hl=pt-PT') # Berthier's page
    
def get_nrc_authors():
    nrc_file = pd.read_json("../nrc/cs-authors-nrc.json")
    nrc_file = nrc_file[['name', 'group_name']]
    groups = nrc_file['group_name'].copy().unique()
    groups.sort()
    return nrc_file

def open_page(url, action):
    page = None
    try:
        sleep_and_build_opener()
        page = opener.open(url) 
        open_allowed_page()
    except URLError as e:     
        print e
        print "URL Error on: " + action
    except Exception as e:     
        print e
        print "Generic Exception on: " + action
    if page != None and page.getcode() != 200:
        print 'F**!!! Response: ' + str(page.getcode() + ' (' + action + ')')
        return None

    return page

def extract_element(page, xpath):
    tree = html.fromstring(page.read())
    return tree.xpath(xpath)

def extract_author_info(row, link, author_page):            
    tree = html.fromstring(author_page.read())

    sc_h_index = tree.xpath("(//td[@class='gsc_rsb_std'])[3]/text()")[0]
    sc_n_cits = tree.xpath("(//td[@class='gsc_rsb_std'])[1]/text()")[0]
    sc_n_cits_2010 = tree.xpath("(//td[@class='gsc_rsb_std'])[2]/text()")[0]
    sc_h_index_2010 = tree.xpath("(//td[@class='gsc_rsb_std'])[4]/text()")[0]

    found_author_scholar = tree.xpath("//div[@id='gsc_prf_in']/text()")
    found_author_scholar = found_author_scholar[0].encode('ascii', 'ignore')

    row.extend((found_author_scholar, sc_h_index, sc_n_cits, sc_h_index_2010, sc_n_cits_2010, link))

### Main

In [21]:
N_AUTHORS = 10
EXCLUDED_LASTNAMES = ["FILHO", "NETO", "JUNIOR", "JR"]
MIN_SLEEP = 2
MAX_RANDOM_MINUTES = 0
AUTHOR_URL_XPATH = "(//h3[@class='gsc_1usr_name']/a)[1]/@href"

start_time = time.time()
opener = build_opener(HTTPCookieProcessor()) 

with open('../h-index/h-index-bot-input.tsv','rb') as input_tsv_file, open('../h-index/h-index-bot-output.tsv', 'wb') as output_tsv:
        
    input_tsv = csv.reader(input_tsv_file, delimiter='\t')
    output_tsv = csv.writer(output_tsv, delimiter='\t')
    rows = []
    i = 0
    k = -1
    not_found = 0
    h_index_not_found = 0
    
    for row in input_tsv:
        if k % 10 == 0: print ("Progress: " + str(k) + " authors (%.1f seg)" % (time.time() - start_time))    
        
        # header
        if (k == -1):
            # header = ('MAS_Found_Author', 'MAS_Id', 'MAS_NPubs', 'MAS_NCits', 'H-index', 'Scholar_Found_Author')
            header = ('GS_Name', 'GS_H5', 'GS_Cits', 'GS_H5_2010', 'GS_Cits_2010', 'GS_Link')
            row.extend(header)
            rows.append(row)
            k = 0
            continue

        k += 1 
        if (k > N_AUTHORS): break

        # Full and Short Names
        full_name = unidecode(row[0].decode('utf-8'))
        names = full_name.split()

        lastname = names[len(names)-1]        
        if (lastname.upper() in EXCLUDED_LASTNAMES and len(names) > 2): lastname = names[len(names)-2]
        short_name = names[0] + ' ' + lastname
        # print "\n" + short_name
        
        # Specific Known Cases
        if full_name == 'Wagner Meira Junior':
            full_name = short_name = 'Wagner Meira'
        elif full_name == 'EDMUNDO ALBUQUERQUE DE SOUZA E SILVA':
            full_name = short_name = 'EDMUNDO SOUZA SILVA'
        
        # University
        university = unidecode(row[4].decode('utf-8'))
        
        # 1st try: Fullname search
        fullname_search_url = 'http://scholar.google.com.br/citations?view_op=search_authors&mauthors=' + urllib.quote(full_name)
        search_page = open_page(fullname_search_url, 'full name search')
        if search_page is None: break

        author_id = extract_element(search_page, AUTHOR_URL_XPATH)

        # 2nd try: Shortname + University search
        if not author_id: 
            shortname_search_url = 'http://scholar.google.com.br/citations?view_op=search_authors&mauthors=' + urllib.quote(short_name + ' ' + university)
            search_page = open_page(shortname_search_url, '(Short name + university) search')
            if search_page is None: break
                
            author_id = extract_element(search_page, AUTHOR_URL_XPATH)
        
        # Giving up, jump to next author        
        if not author_id: 
            h_index_not_found += 1
        else: 
            author_id = author_id[0]
            author_url = "https://scholar.google.com.br" + author_id
            author_page = open_page(author_url, 'opening author profile')
            if author_page is None: break
            
            extract_author_info(row, author_id, author_page)
     
        # Add row with author info or an empty row if it was not found
        rows.append(row)
    
    # end for 
    output_tsv.writerows(rows)

if (k != 1):
    print("Authors Not Found in GSC: %d of %d (%.1f%%)" % (h_index_not_found, k-1, (h_index_not_found * 100.00) / (k-1)))
print("\n--- Finished after %.3f seconds. ---" % (time.time() - start_time))

del opener


Progress: 0 authors (0.0 seg)
Progress: 10 authors (154.9 seg)
Authors Not Found in GSC: 2 of 10 (20.0%)

--- Finished after 154.898 seconds. ---


### MAS robot (not used)

In [ ]:
        try:
            page = requests.get('http://academic.research.microsoft.com/Search?query=author:(' + full_name + ')')
        except:     
            print "[1] Erro de conexão, prosseguindo..."
            continue

        tree = html.fromstring(page.text)

        # Finding author
        # At least three matched names
        link = tree.xpath("(//a[@class='author-name-tooltip']//b[3])[1]/../@href")

        if not link: 
            try:
                page = requests.get('http://academic.research.microsoft.com/Search?query=author:(' + short_name + ')')
            except:     
                print "[2] Erro de conexão, prosseguindo..."
                continue
            
            tree = html.fromstring(page.text)
            link = tree.xpath("(//a[@class='author-name-tooltip']//b[2])[1]/../@href")

        if not link: 
            link = tree.xpath("(//a[@class='author-name-tooltip']//b[1])[1]/../@href")

        if not link: 
            not_found += 1
            # rows.append(row)
            # output_tsv.writerow(row)
            # continue;
        else:
            link = link[0]

            # mas_id
            url_parts = link.split('/')
            mas_id = url_parts[len(url_parts)-2]
            found_author = url_parts[len(url_parts)-1]

            # cit and pub
            try:
                page = requests.get(link)
            except:     
                print "[3] Erro de conexão, prosseguindo..."
                continue
                
            tree = html.fromstring(page.text)

            n_pub = tree.xpath('//div[@class="author-card"]//a[@id="ctl00_MainContent_AuthorItem_publication"]/span/text()');
            if not n_pub: n_pub = 0
            else: n_pub = n_pub[0]         

            n_cit = tree.xpath('//div[@class="author-card"]//a[@id="ctl00_MainContent_AuthorItem_citedBy"]/span/text()');    
            if not n_cit: n_cit = 0
            else: n_cit = n_cit[0]

            row.extend((found_author, mas_id, n_pub, n_cit))

## NRC Json

In [36]:
import pandas as pd
import numpy as np


array([u'<<< artificial reference group >>>', u'Arizona State University',
       u'Auburn University', u'Boston University', u'Brandeis University',
       u'Brigham Young University', u'Brown University',
       u'California Institute of Technology',
       u'Carnegie Mellon University', u'Case Western Reserve University',
       u'City University of New York Grad. Center', u'Clemson University',
       u'College of William and Mary',
       u'Columbia University In The City of New York',
       u'Cornell University', u'Dartmouth College', u'Duke University',
       u'ETH Zurich', u'Florida Institute of Technology',
       u'Florida International University', u'Florida State University',
       u'George Washington University', u'Georgia Institute of Technology',
       u'Georgia State University', u'Harvard University',
       u'Illinois Institute of Technology', u'Imperial College London',
       u'Indiana University at Bloomington', u'Iowa State University',
       u'Johns Hopkins 

In [ ]:

# %timeit x = 'a'

# for x in nrc_file['name'].tolist():
    # print x
# cnpq_authors = cnpq_authors.ix[:, ['Name', 'ShortName', 'CNPqLevel']]
# fullnames_list = [unidecode(x.decode(UTF8)).upper() for x in cnpq_authors['Name'].tolist()]
# shortnames_list = [unidecode(x.decode(UTF8)).upper() for x in cnpq_authors['ShortName'].tolist()]